- LLMChain can be used to take a prompt template, pass it to an llm with input and get a response all in 1 step
- Sequential chains run sequence of chains 1 after another. works best when we have chains that expect only one input and return only 1 output
Input
  |
  v
+-------------------+
|   First Chain     |   (e.g., generates a company name from a product)
+-------------------+
  |
  v
+-------------------+
|   Second Chain    |   (e.g., writes a description for the company name)
+-------------------+
  |
  v
Output
- when there are multiple inputs/outputs you can use the regular sequential chain but the whole chain which is a combination of multiple chains has a single input & output.
- the important thing here is the naming of inputs & outputs. You need to get the right so that subsequent chains take in the correct input
         +-------------------+
Inputs:  |  {"product": ...,
         |   "country": ...} |
         +-------------------+
                   |
                   v
        +--------------------------+
        |      First Chain         |
        |  (e.g., uses product &   |
        |   country, outputs       |
        |   company_name & slogan) |
        +--------------------------+
                   |
                   v
        +--------------------------+
        |      Second Chain        |
        |  (e.g., uses company_name|
        |   & slogan, outputs      |
        |   description)           |
        +--------------------------+
                   |
                   v
         +-------------------+
Outputs: | {"company_name": ...,
         |  "slogan": ...,
         |  "description": ...}
         +-------------------+

In [1]:
import os

from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv())  # read local .env

In [3]:
#!pip install pandas

   ---------------------------------------- 0.0/11.0 MB ? eta -:--:--
   - -------------------------------------- 0.5/11.0 MB 4.2 MB/s eta 0:00:03
   ----- ---------------------------------- 1.6/11.0 MB 4.0 MB/s eta 0:00:03
   --------- ------------------------------ 2.6/11.0 MB 4.3 MB/s eta 0:00:02
   ------------ --------------------------- 3.4/11.0 MB 4.4 MB/s eta 0:00:02
   ---------------- ----------------------- 4.5/11.0 MB 4.6 MB/s eta 0:00:02
   -------------------- ------------------- 5.5/11.0 MB 4.6 MB/s eta 0:00:02
   ----------------------- ---------------- 6.6/11.0 MB 4.7 MB/s eta 0:00:01
   --------------------------- ------------ 7.6/11.0 MB 4.7 MB/s eta 0:00:01
   ------------------------------ --------- 8.4/11.0 MB 4.6 MB/s eta 0:00:01
   --------------------------------- ------ 9.2/11.0 MB 4.4 MB/s eta 0:00:01
   ------------------------------------ --- 10.0/11.0 MB 4.4 MB/s eta 0:00:01
   ---------------------------------------  10.7/11.0 MB 4.3 MB/s eta 0:00:01
   -


[notice] A new release of pip is available: 24.3.1 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [4]:
import pandas as pd

#create sample dataframe
data = {
    'name': ['nicole', 'karl', 'marion'],
    'age': [25, 14, 50],
    'city': ['galway', 'athlone', 'london']
}
df = pd.DataFrame(data)
df.to_csv('data.csv', index=False)

In [5]:
df = pd.read_csv('data.csv')

In [6]:
df.head()

,name,age,city
0,nicole,25,galway
1,karl,14,athlone
2,marion,50,london


## LLMChain

In [7]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain.chains import LLMChain

In [8]:
# Add a little randomness in temp for creativity
llm = ChatOpenAI(temperature=0.9)

C:\Users\colgann\AppData\Local\Temp\ipykernel_37824\4147887815.py:2: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import ChatOpenAI``.
  llm = ChatOpenAI(temperature=0.9)


In [9]:
prompt = ChatPromptTemplate.from_template(
    "What is the best name to describe a company that makes {product}?"
)

In [10]:
chain = LLMChain(llm=llm,prompt=prompt)

C:\Users\colgann\AppData\Local\Temp\ipykernel_37824\2891286404.py:1: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use :meth:`~RunnableSequence, e.g., `prompt | llm`` instead.
  chain = LLMChain(llm=llm,prompt=prompt)


In [11]:
product = "Phones"
chain.run(product)

C:\Users\colgann\AppData\Local\Temp\ipykernel_37824\818320379.py:2: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  chain.run(product)


'"TechTalk"'

## SimpleSequentialChain

In [12]:
from langchain.chains import SimpleSequentialChain

In [14]:
llm = ChatOpenAI(temperature=0.9)

# Prompt template 1
first_prompt = ChatPromptTemplate.from_template(
    "What is the best name to describe a company that makes {product}?"
)

# chain 1
first_chain = LLMChain(llm=llm, prompt=first_prompt)

In [15]:
# Prompt template 2
second_prompt = ChatPromptTemplate.from_template(
    "Write a 20 word description for the following company: {company_name}"
)
#chain 2
second_chain = LLMChain(llm=llm,prompt=second_prompt)

In [16]:
overall_simple_chain = SimpleSequentialChain(
    chains=[first_chain, second_chain],
    verbose=True
)

In [17]:
overall_simple_chain.run("protein powder")



> Entering new SimpleSequentialChain chain...
"Protein Fusion"
Protein Fusion creates innovative and delicious protein bars that blend plant-based and whey proteins for the ultimate post-workout snack.

> Finished chain.


'Protein Fusion creates innovative and delicious protein bars that blend plant-based and whey proteins for the ultimate post-workout snack.'

## SequentialChain

In [23]:
from langchain.chains import SequentialChain

In [19]:
llm = ChatOpenAI(temperature=0.9)

# Prompt template 1: translate to english
first_prompt = ChatPromptTemplate.from_template(
    "Translate the following review to english" \
    "\n\n{review}"
)

# Chain1: input=review, output=english_review
chain_one = LLMChain(llm=llm, prompt=first_prompt, output_key="english_review")

In [20]:
second_prompt = ChatPromptTemplate.from_template(
    "Summarise the following review in 1 sentence:" \
    "\n\n{english_review}"
)
#chain2: input=english_review, output=summary
chain_two = LLMChain(llm=llm, prompt=second_prompt, output_key="summary")

In [21]:
# Prompt template3: find out the language of the review
third_prompt = ChatPromptTemplate.from_template(
    "What language is the following review:" \
    "\n\n{review}"
)
# Chain3: input=initial review, output=language
chain_three = LLMChain(llm=llm, prompt=third_prompt, output_key="language")

In [22]:
# Prompt4: follow up message
fourth_prompt = ChatPromptTemplate.from_template(
    "Write a follow up response to the following"
    "summary in the specified language"
    "\n\n{summary}\n\n{language}"
)
#chain4: input=summary,language, output=followup_msg
chain_four = LLMChain(llm=llm, prompt=fourth_prompt, output_key="followup_msg")

In [24]:
# Overall_chain: input=review, output=english_review, summary, followup_msd
overall_chain = SequentialChain(
    chains=[chain_one,chain_two,chain_three,chain_four],
    input_variables=["review"],
    output_variables=["english_review", "summary", "followup_msg"],
    verbose=True
)

In [25]:
review = "Ich liebe es!"
overall_chain(review)

C:\Users\colgann\AppData\Local\Temp\ipykernel_37824\2309184924.py:2: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  overall_chain(review)




> Entering new SequentialChain chain...

> Finished chain.


{'review': 'Ich liebe es!',
 'english_review': 'I love it!',
 'summary': 'The reviewer has a strong positive feeling towards the product or experience.',
 'followup_msg': 'Vielen Dank für Ihr tolles Feedback! Es freut uns sehr zu hören, dass Sie so eine positive Erfahrung mit unserem Produkt / unserer Dienstleistung gemacht haben. Wir sind immer bestrebt, unseren Kunden das Beste zu bieten und es macht uns sehr glücklich zu hören, dass es Ihnen gefallen hat. Wir hoffen, Sie auch in Zukunft weiterhin zufriedenstellen zu können. Vielen Dank nochmals für Ihr Lob!'}

In [ ]:
chains 7:00